Given code: 
1. import packages
2. load image pairs img1 & img2

In [16]:
import cv2
import numpy as np

# Load image pairs - you can change file names
image1 = cv2.imread("ings\IMG_7577.png")
image2 = cv2.imread("ings\IMG_7578.png")

Problem 1: [code by yourself]

Define a function `get_transform_from_keypoints()` that computes the 3x3 homogeneous transform H from keypoint matches with the following input parameters and return variables:
* img_src (input): image that is warped to be aligned to img_dst 
* img_dst (input): reference image to align img_src
* H (output/return): computed linear transform matrix
* kpts_src (output/return): computed keypoints for img_src
* dscrpt_src (output/return): computed descriptors for img_src
* kpts_dst (output/return): computed keypoints for img_dst
* dscrpt_dst (output/return): computed descriptors for img_dst
* matches (output/return): keypoint matches determined from SIFT

When computing the linear transform, follow this proceses:
* detect SIFT keypoints and compute SIFT descriptors
* find the linear transform matrix H using the matched keypoints

In [47]:
def get_transform_from_keypoints(img_src, img_dst):
  # Create a SIFT object and detect keypoints and descriptors for each image
  sift=cv2.SIFT_create(nOctaveLayers=10)
  kpts_src,dscrpt_src=sift.detectAndCompute(img_src,None)
  kpts_dst,dscrpt_dst=sift.detectAndCompute(img_dst,None)

  # Match the descriptors
  matcher = cv2.BFMatcher()
  # src기준으로 dst에서 가장 가까운 이웃과 그 다음으로 가까운 이웃을 반환한다(k=2).
  matches = matcher.knnMatch(dscrpt_src, dscrpt_dst, k=2)  

  # 두번째 distance와 ratio의 곱보다 첫번쨰 distance가 작으면 good match
  # ratio가 작아질수록 첫번째 distance가 압도적으로 적어 매칭관련성이 매우 높다는것을 뜻한다.
  good_matches = []
  ratio=0.12
  for m, n in matches:
      if m.distance < ratio * n.distance:  
          good_matches.append(m)
  # print(good_matches[0].distance,good_matches[1].distance)
  # good_matches에는 좋은 매칭들이 포함되어 있다
  
  # 좋은 매칭에 대해 queryIdx와 trainIdx를 사용하여 원본 이미지와 대상 이미지의 키포인트 좌표를 추출
  src_pts = np.float32([kpts_src[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  dst_pts = np.float32([kpts_dst[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  
  print(src_pts)
  print(dst_pts)
  # Find the homography matrix using the matched keypoints
  H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)

  # return outputs
  return H, kpts_src, dscrpt_src, kpts_dst, dscrpt_dst, good_matches

# image1=cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
# image2=cv2.cvtColor(image2,cv2.COLOR_BGR2GRAY)

H, kpts_src, dscrpt_src, kpts_dst, dscrpt_dst, matches=get_transform_from_keypoints(image1,image2)

[[[533.8938  589.7643 ]]

 [[603.7789  585.0748 ]]

 [[703.61804 712.7682 ]]

 [[738.87823 498.10538]]

 [[739.9495  589.2371 ]]

 [[793.863   547.1412 ]]]
[[[300.97025 497.69577]]

 [[369.88748 510.37115]]

 [[429.61197 652.71844]]

 [[524.2998  460.36768]]

 [[499.8654  547.071  ]]

 [[563.82556 520.0668 ]]]


In [48]:
def draw_matching_results(img_src, img_dst, kpts_src, kpts_dst, matches):
    # Draw the matches between the source and destination images
    matched_image = cv2.drawMatches(img_src, kpts_src, img_dst, kpts_dst, matches, None)
    #matched_image=cv2.resize(matched_image,(matched_image.shape[1]//2,matched_image.shape[0]//2))

    # Display the image with the matches
    cv2.imshow('Matching Results', matched_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
draw_matching_results(image1,image2,kpts_src,kpts_dst,matches)

Problem 2: [Code by yourself]

Define a function named `stitch_image()` that generates a stitched image from img_ref, img_align, and T, with the following input parameters and return variables:
* img_src (input): image that is warped to be aligned to img_dst 
* img_dst (input): reference image to align img_src
* H (input): computed linear transform matrix
* stitched_image (output): the stitched image

This function should include the following processes:
* Compute the size of the output stitched image and the offset that ensures all pixels from both images are included in the stitched image
* Modify H to account for the image offset
* Warp the first src image into the second image
* Blend in the second dst image

In [ ]:
def get_stitched_image(img_src, img_dst, H):
  # Compute the size of the output stitched image
  ### CODE YOURSELF ###

  
  # Modify H to account for the image offset
  ### CODE YOURSELF ###

  # Warp the first image to the perspective of the second image
  ### CODE YOURSELF ###

  # Combine the two images to create a single stitched image
  ### CODE YOURSELF ###

  # return output
  return stitched_image

Given code: 
3. Call function `get_transform_from_keypoints` - detect and match keypoints and compute transform
4. Draw the matches on a new image to check validity of matched keypoints
5. Call function `get_stitched_image` - create stitched image and save

In [ ]:
# 3. Detect and match keypoints and compute transform
H, kpts_src, _, kpts_dst, _, matches = get_transform_from_keypoints(image1, image2)

# 4. Draw the matches on a new image to check validity of matched keypoints
match_image = cv2.drawMatches(image1, kpts_src, image2, kpts_dst, matches, None)
cv2.imwrite('matches.png', match_image)

# 3. Create stitched image and save
stitched_image = get_stitched_image(image1, image2, H)
cv2.imwrite('stitched.png', stitched_image)